# Day 5

## Part 1

In [1]:
# lines = [line.rstrip() for line in open('day5_input.txt')]
lines = [line.rstrip() for line in open('day5_sample.txt')]

In [2]:
def split_vals(string):
    return [int(number) for number in string.split(' ') if number.isdigit()]

In [3]:
categories = [(line.split(':')[0], lines.index(line)) for line in lines if ':' in line][1:]
initial_seeds = split_vals(lines[0].split(':')[1])

map_names = []
map_values = []
for i in range(len(categories)):
    start_index = categories[i][1] + 1
    end_index = categories[i+1][1] if i!=len(categories)-1 else -1
    maps = lines[start_index:categories[i+1][1]] if i!=len(categories)-1 else lines[start_index:]
    maps = [line for line in maps if line!='']
    maps = [split_vals(map) for map in maps]
    map_names.append(categories[i][0])
    map_values.append(maps)

In [4]:
destination_range_starts = [[i[0] for i in sublist] for sublist in map_values]
source_range_starts = [[i[1] for i in sublist] for sublist in map_values]
range_lengths = [[i[-1] for i in sublist] for sublist in map_values]

## NAIVE SOLUTION

## IMPROVED SOLUTION

In [5]:
maps = []

for destination_starts, source_starts, ranges in zip(destination_range_starts, source_range_starts, range_lengths):
    map_dict = {}
    for destination_start, source_start, r in zip(destination_starts, source_starts, ranges):
        map_dict[source_start] = (destination_start, r)
    maps.append(map_dict)

[seeds, soil, fertilizer, water, light, temperature, humidity] = maps

In [6]:
map_names

['seed-to-soil map',
 'soil-to-fertilizer map',
 'fertilizer-to-water map',
 'water-to-light map',
 'light-to-temperature map',
 'temperature-to-humidity map',
 'humidity-to-location map']

In [7]:
def lookup(dictionary, i):
    # find the largest key less than or equal to i
    l = [key for key in dictionary if key <= i]
    if len(l) == 0:
        return i
    the_key = max(l)
    mapping_start, mapping_len = dictionary[the_key]
    mapping_offset = i - the_key
    if mapping_offset >= mapping_len:
        return i
    return mapping_start + mapping_offset

def lookup_location(i):
    soil_val = lookup(seeds, i)
    fertilizer_val = lookup(soil, soil_val)
    water_val = lookup(fertilizer, fertilizer_val)
    light_val = lookup(water, water_val)
    temp_val = lookup(light, light_val)
    humidity_val = lookup(temperature, temp_val)
    return lookup(humidity, humidity_val)

In [8]:
locations = [lookup_location(x) for x in initial_seeds]

min(locations)

35